In [7]:
from opensearchpy import OpenSearch

client = OpenSearch("http://localhostL9200", use_ssl=False)

In [8]:
query = {
               "query": {
                 "match_all": {}
               }
             }

In [10]:
response = client.search(
    body=query,
    index='ecommerce'
)

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x106017e00>: Failed to resolve 'localhostl9200' ([Errno 8] nodename nor servname provided, or not known)) caused by: NameResolutionError(<urllib3.connection.HTTPConnection object at 0x106017e00>: Failed to resolve 'localhostl9200' ([Errno 8] nodename nor servname provided, or not known))